In [ ]:
# Generalized Autoregressive Conditional Heteroskedasticity (GARCH) Model Main Flow
import numpy as np
import pandas as pd
from arch import arch_model
from statsmodels.tsa.stattools import acf, adfuller
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import shapiro

# -----------------------------
# 1. Generate Random Time-Series Data
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Initialize GARCH(1,1) Model & Estimate Parameters
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# Extract parameters (MLE & QMLE)
mle_params = garch_results.params
q_mle_params = garch_results.param_cov  

print("\n📌 Parameter Estimation:")
print("   - MLE Parameters:\n", mle_params)
print("   - QMLE Covariance Matrix:\n", q_mle_params)

# -----------------------------
# 3. Model Diagnostics
# -----------------------------
acf_values = acf(garch_results.resid.dropna(), fft=False)
ljung_box_test = acorr_ljungbox(garch_results.resid.dropna(), lags=[10], return_df=True)
adf_test = adfuller(garch_results.resid.dropna())

print("\n📌 Model Diagnostics:")
print("   - ACF Values:\n", acf_values)
print("   - Ljung-Box Test:\n", ljung_box_test)
print(f"   - Augmented Dickey-Fuller Test: p-value = {adf_test[1]:.5f}")

# -----------------------------
# 4. Forecasting Future Volatility
# -----------------------------
forecast = garch_results.forecast(horizon=5)
one_step_forecast = forecast.variance.iloc[-1, 0]
multi_step_forecast = forecast.variance.iloc[-1, :]

print("\n📌 Forecasting Future Volatility:")
print(f"   - One-Step Ahead Forecast: {one_step_forecast:.5f}")
print(f"   - Multi-Step Ahead Forecast: {multi_step_forecast.tolist()}")

# -----------------------------
# 5. Residual Analysis
# -----------------------------
dw_stat = np.sum(np.diff(garch_results.resid.dropna())**2) / np.sum(garch_results.resid.dropna()**2)
jb_stat, jb_pvalue, skewness, kurtosis = jarque_bera(garch_results.resid.dropna())  # Fixed unpacking
shapiro_stat, shapiro_pvalue = shapiro(garch_results.resid.dropna())

print("\n📌 Residual Analysis:")
print(f"   - Durbin-Watson Statistic: {dw_stat:.5f}")
print(f"   - Jarque-Bera Test p-value: {jb_pvalue:.5f}")
print(f"   - Shapiro-Wilk Test p-value: {shapiro_pvalue:.5f}")

# -----------------------------
# 6. Stability Analysis
# -----------------------------
roots = np.roots([1] + [-mle_params['omega'], -mle_params['alpha[1]'], -mle_params['beta[1]']])
mean_reversion_time = -1 / np.log(np.abs(roots))

print("\n📌 Stability Analysis:")
print("   - Characteristic Roots:", roots)
print("   - Mean Reversion Time:", mean_reversion_time)

# -----------------------------
# 7. Impulse Response Function (IRF)
# -----------------------------
irf_steps = 10
irf_values = np.array([garch_results.forecast(horizon=i + 1).variance.iloc[-1, -1] for i in range(irf_steps)])

print("\n📌 Impulse Response Function:")
print("   - IRF Values:", irf_values)

# -----------------------------
# 8. Model Selection Criteria (AIC, BIC)
# -----------------------------
aic = garch_results.aic
bic = garch_results.bic

print("\n📌 Model Selection Criteria:")
print(f"   - AIC: {aic:.5f}")
print(f"   - BIC: {bic:.5f}")

# -----------------------------
# FINAL DECISION
# -----------------------------
print("\n✅ GARCH Model Analysis Completed!\n")


📌 Parameter Estimation:
   - MLE Parameters:
 mu          1.824052e-02
omega       2.336675e-02
alpha[1]    2.797593e-08
beta[1]     9.757849e-01
Name: params, dtype: float64
   - QMLE Covariance Matrix:
                     mu     omega      alpha[1]   beta[1]
mu        9.600929e-04  0.000059  4.266182e-07 -0.000059
omega     5.886868e-05  0.000509 -2.644983e-05 -0.000501
alpha[1]  4.266182e-07 -0.000026  8.215964e-06  0.000020
beta[1]  -5.908906e-05 -0.000501  1.965137e-05  0.000501

📌 Model Diagnostics:
   - ACF Values:
 [ 1.00000000e+00 -7.47776769e-03 -2.32989804e-04  1.61719885e-02
 -5.42323823e-02  2.58165932e-02  1.43284233e-02  1.22522388e-02
 -4.90697469e-02  2.73701884e-02  1.45333933e-02  3.06797960e-02
  1.40742520e-02  3.35282053e-04 -1.33175733e-02 -9.93517628e-03
  7.17566784e-03 -2.54215627e-02  3.25084227e-02 -6.18979393e-03
 -6.39429710e-02 -1.91792138e-02  2.43170232e-02  3.77688611e-02
 -1.88914059e-03 -2.92551605e-02  1.30470297e-02  1.36710424e-02
  1.05333919e-

In [6]:
# Generalized Autoregressive Conditional Heteroskedasticity (GARCH) Model Derivative Flows
# Flow 1: Parameter Estimation
import numpy as np
import pandas as pd
from arch import arch_model

# -----------------------------
# 1. Generate Random Time-Series Data (Same as Main Flow)
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Initialize & Fit GARCH(1,1) Model
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# -----------------------------
# 3. Extract Parameter Estimations (MLE & QMLE)
# -----------------------------
mle_params = garch_results.params  # Maximum Likelihood Estimates
q_mle_cov_matrix = garch_results.param_cov  # Quasi-MLE Covariance Matrix

# -----------------------------
# 4. Decision Rule (Based on Significance & Stability)
# -----------------------------
alpha = 0.05  # Significance level
p_values = garch_results.pvalues
significance_decision = {
    param: "Significant" if p < alpha else "Not Significant"
    for param, p in p_values.items()
}

# Stability Check: GARCH(1,1) is stable if (alpha + beta < 1)
alpha_coeff = mle_params.get('alpha[1]', 0)
beta_coeff = mle_params.get('beta[1]', 0)
stability_condition = alpha_coeff + beta_coeff < 1
stability_decision = "Stable Model" if stability_condition else "Unstable Model"

# -----------------------------
# 5. Print Results & Decisions
# -----------------------------
print("\n📌 Parameter Estimation (Flow 1):")
print("   - MLE Parameters:\n", mle_params)
print("   - QMLE Covariance Matrix:\n", q_mle_cov_matrix)
print("   - Parameter Significance:\n", significance_decision)
print(f"   - Model Stability Condition (α + β < 1): {stability_condition}")
print(f"   - Final Decision: {stability_decision}")

print("\n✅ Flow 1: Parameter Estimation Completed!\n")


📌 Parameter Estimation (Flow 1):
   - MLE Parameters:
 mu          1.824052e-02
omega       2.336675e-02
alpha[1]    2.797593e-08
beta[1]     9.757849e-01
Name: params, dtype: float64
   - QMLE Covariance Matrix:
                     mu     omega      alpha[1]   beta[1]
mu        9.600929e-04  0.000059  4.266182e-07 -0.000059
omega     5.886868e-05  0.000509 -2.644983e-05 -0.000501
alpha[1]  4.266182e-07 -0.000026  8.215964e-06  0.000020
beta[1]  -5.908906e-05 -0.000501  1.965137e-05  0.000501
   - Parameter Significance:
 {'mu': 'Not Significant', 'omega': 'Not Significant', 'alpha[1]': 'Not Significant', 'beta[1]': 'Significant'}
   - Model Stability Condition (α + β < 1): True
   - Final Decision: Stable Model

✅ Flow 1: Parameter Estimation Completed!



In [7]:
# Flow 2: Model Diagnostics
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf
from statsmodels.stats.diagnostic import acorr_ljungbox
from arch import arch_model

# -----------------------------
# 1. Generate Random Time-Series Data (Same as Main Flow)
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Fit GARCH(1,1) Model (Refers to Main Flow)
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# -----------------------------
# 3. Compute ACF for Residuals
# -----------------------------
residuals = garch_results.resid.dropna()
acf_values = acf(residuals, fft=False, nlags=20)

# -----------------------------
# 4. Ljung-Box Test (White Noise Check)
# -----------------------------
lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
lb_pvalue = lb_test['lb_pvalue'].values[0]

# Decision Rule for Ljung-Box:
lb_decision = "Residuals are White Noise" if lb_pvalue > 0.05 else "Residuals are NOT White Noise"

# -----------------------------
# 5. Augmented Dickey-Fuller (ADF) Test (Stationarity Check)
# -----------------------------
adf_stat, adf_pvalue, _, _, _, _ = adfuller(residuals)

# Decision Rule for ADF Test:
adf_decision = "Residuals are Stationary" if adf_pvalue < 0.05 else "Residuals are NOT Stationary"

# -----------------------------
# 6. Standard Errors & Confidence Intervals
# -----------------------------
std_errors = garch_results.std_err
conf_intervals = garch_results.conf_int()

# -----------------------------
# 7. Print Results & Decision
# -----------------------------
print("\n📌 Model Diagnostics (Flow 2):")
print("   - ACF Values:\n", acf_values)
print("   - Ljung-Box Test p-value:", lb_pvalue)
print(f"   - Ljung-Box Decision: {lb_decision}")
print("   - ADF Test p-value:", adf_pvalue)
print(f"   - ADF Decision: {adf_decision}")
print("   - Standard Errors:\n", std_errors)
print("   - Confidence Intervals:\n", conf_intervals)

print("\n✅ Flow 2: Model Diagnostics Completed!\n")


📌 Model Diagnostics (Flow 2):
   - ACF Values:
 [ 1.00000000e+00 -7.47776769e-03 -2.32989804e-04  1.61719885e-02
 -5.42323823e-02  2.58165932e-02  1.43284233e-02  1.22522388e-02
 -4.90697469e-02  2.73701884e-02  1.45333933e-02  3.06797960e-02
  1.40742520e-02  3.35282053e-04 -1.33175733e-02 -9.93517628e-03
  7.17566784e-03 -2.54215627e-02  3.25084227e-02 -6.18979393e-03
 -6.39429710e-02]
   - Ljung-Box Test p-value: 0.6570611936707339
   - Ljung-Box Decision: Residuals are White Noise
   - ADF Test p-value: 0.0
   - ADF Decision: Residuals are Stationary
   - Standard Errors:
 mu          0.030985
omega       0.022559
alpha[1]    0.002866
beta[1]     0.022383
Name: std_err, dtype: float64
   - Confidence Intervals:
              lower     upper
mu       -0.042490  0.078971
omega    -0.020848  0.067581
alpha[1] -0.005618  0.005618
beta[1]   0.931916  1.019654

✅ Flow 2: Model Diagnostics Completed!



In [ ]:
# Flow 3: Forecasting Future Volatility
import numpy as np
import pandas as pd
from arch import arch_model

# -----------------------------
# 1. Generate Random Time-Series Data (Same as Main Flow)
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Fit GARCH(1,1) Model (Refers to Main Flow)
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# -----------------------------
# 3. One-Step Ahead Forecasting (Fix Applied)
# -----------------------------
one_step_forecast = garch_results.forecast(start=len(returns_series) - 1, horizon=1)
one_step_volatility = np.sqrt(one_step_forecast.variance.iloc[0, 0])  # Extract first forecasted variance

# -----------------------------
# 4. Multi-Step Ahead Forecasting (10 Steps)
# -----------------------------
n_steps = 10  # Forecast horizon
multi_step_forecast = garch_results.forecast(start=len(returns_series) - 1, horizon=n_steps)
multi_step_volatility = np.sqrt(multi_step_forecast.variance.iloc[0])  # Extract multi-step volatility

# -----------------------------
# 5. Compute Prediction Interval (95% Confidence)
# -----------------------------
alpha = 0.05  # Significance level
z_critical = 1.96  # Approximate z-score for 95% confidence

multi_step_upper = multi_step_volatility + z_critical * multi_step_volatility.std()
multi_step_lower = multi_step_volatility - z_critical * multi_step_volatility.std()

# -----------------------------
# 6. Compute Forecast Errors & Confidence Interval (Fix Applied)
# -----------------------------
forecast_errors = returns_series[-n_steps:] - multi_step_volatility.mean()  # Actual vs Forecasted
forecast_error_std = forecast_errors.std()

error_upper = forecast_errors.mean() + z_critical * forecast_error_std
error_lower = forecast_errors.mean() - z_critical * forecast_error_std  # ✅ FIX: Removed parentheses

# -----------------------------
# 7. Print Results & Decision
# -----------------------------
print("\n📌 Forecasting Future Volatility (Flow 3):")
print("   - One-Step Ahead Forecasted Volatility:", one_step_volatility)
print("   - Multi-Step Ahead Forecasted Volatility:\n", multi_step_volatility)
print("   - 95% Prediction Interval:\n   Lower:", multi_step_lower, "\n   Upper:", multi_step_upper)
print("   - Forecast Errors:\n", forecast_errors)
print("   - Forecast Error Confidence Interval:\n   Lower:", error_lower, "\n   Upper:", error_upper)

print("\n✅ Flow 3: Forecasting Future Volatility Completed!\n")


📌 Forecasting Future Volatility (Flow 3):
   - One-Step Ahead Forecasted Volatility: 0.9823265208842523
   - Multi-Step Ahead Forecasted Volatility:
 h.01    0.982327
h.02    0.982327
h.03    0.982327
h.04    0.982327
h.05    0.982327
h.06    0.982327
h.07    0.982327
h.08    0.982327
h.09    0.982327
h.10    0.982327
Name: 999, dtype: float64
   - 95% Prediction Interval:
   Lower: h.01    0.982327
h.02    0.982327
h.03    0.982327
h.04    0.982327
h.05    0.982327
h.06    0.982327
h.07    0.982327
h.08    0.982327
h.09    0.982327
h.10    0.982327
Name: 999, dtype: float64 
   Upper: h.01    0.982327
h.02    0.982327
h.03    0.982327
h.04    0.982327
h.05    0.982327
h.06    0.982327
h.07    0.982327
h.08    0.982327
h.09    0.982327
h.10    0.982327
Name: 999, dtype: float64
   - Forecast Errors:
 990   -0.773944
991   -3.024061
992   -1.229504
993   -1.664311
994   -1.983947
995   -1.263427
996    0.815360
997   -0.341484
998   -1.553506
999   -0.409744
Name: Returns, dtype: float

In [ ]:
# Flow 4: Residual Analysis
import numpy as np
import pandas as pd
from arch import arch_model
from statsmodels.stats.stattools import durbin_watson, jarque_bera

# -----------------------------
# 1. Generate Random Time-Series Data (Same as Main Flow)
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Fit GARCH(1,1) Model (Refers to Main Flow)
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# -----------------------------
# 3. Extract Residuals from GARCH Model
# -----------------------------
residuals = garch_results.resid.dropna()  # Remove NaN values

# -----------------------------
# 4. Perform Durbin-Watson Test (Check for Autocorrelation in Residuals)
# -----------------------------
dw_stat = durbin_watson(residuals)

# -----------------------------
# 5. Perform Jarque-Bera Test (Check for Normality of Residuals)
# -----------------------------
jb_stat, jb_pvalue, skewness, kurtosis = jarque_bera(residuals)

# -----------------------------
# 6. Print Results & Decision
# -----------------------------
print("\n📌 Residual Analysis (Flow 4):")
print("   - Durbin-Watson Statistic:", dw_stat)
if dw_stat < 1.5:
    print("   → Possible Positive Autocorrelation.")
elif dw_stat > 2.5:
    print("   → Possible Negative Autocorrelation.")
else:
    print("   → No Significant Autocorrelation.")

print("\n   - Jarque-Bera Statistic:", jb_stat)
print("   - p-value:", jb_pvalue)
print("   - Skewness:", skewness)
print("   - Kurtosis:", kurtosis)
if jb_pvalue < 0.05:
    print("   → Residuals are NOT normally distributed (Reject H0).")
else:
    print("   → Residuals are normally distributed (Fail to Reject H0).")

print("\n✅ Flow 4: Residual Analysis Completed!\n")


📌 Residual Analysis (Flow 4):
   - Durbin-Watson Statistic: 2.014395584135408
   → No Significant Autocorrelation.

   - Jarque-Bera Statistic: 2.456373201879963
   - p-value: 0.2928231016469249
   - Skewness: 0.11680083110533511
   - Kurtosis: 3.0662058929214844
   → Residuals are normally distributed (Fail to Reject H0).

✅ Flow 4: Residual Analysis Completed!



In [12]:
# Flow 5: Stability Analysis
import numpy as np
import pandas as pd
from arch import arch_model

# -----------------------------
# 1. Generate Random Time-Series Data (Same as Main Flow)
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Fit GARCH(1,1) Model (Refers to Main Flow)
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# -----------------------------
# 3. Compute Characteristic Roots (Check Model Stability)
# -----------------------------
params = garch_results.params
alpha_1 = params.get("alpha[1]", np.nan)  # ARCH term
beta_1 = params.get("beta[1]", np.nan)  # GARCH term
sum_alpha_beta = alpha_1 + beta_1

# -----------------------------
# 4. Compute Mean Reversion Time (Time Taken to Return to Mean)
# -----------------------------
if beta_1 < 1:
    mean_reversion_time = -1 / np.log(beta_1)  # Formula: -1 / log(beta)
else:
    mean_reversion_time = np.inf  # Non-mean-reverting

# -----------------------------
# 5. Print Results & Decision
# -----------------------------
print("\n📌 Stability Analysis (Flow 5):")

print("   - Characteristic Root (α₁ + β₁):", sum_alpha_beta)
if sum_alpha_beta < 1:
    print("   → Model is Stable ✅")
else:
    print("   → Model is Unstable ❌")

print("\n   - Mean Reversion Time:", mean_reversion_time)
if mean_reversion_time == np.inf:
    print("   → Model is Non-Mean-Reverting ❌")
else:
    print("   → Model is Mean-Reverting ✅")

print("\n✅ Flow 5: Stability Analysis Completed!\n")


📌 Stability Analysis (Flow 5):
   - Characteristic Root (α₁ + β₁): 0.9757848792598038
   → Model is Stable ✅

   - Mean Reversion Time: 40.794420546729555
   → Model is Mean-Reverting ✅

✅ Flow 5: Stability Analysis Completed!



In [13]:
# Flow 6: Impulse Response Function
import numpy as np
import pandas as pd
from arch import arch_model

# -----------------------------
# 1. Generate Random Time-Series Data (Same as Main Flow)
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Fit GARCH(1,1) Model (Refers to Main Flow)
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# -----------------------------
# 3. Compute Impulse Response Function (Response to Shocks)
# -----------------------------
params = garch_results.params
alpha_1 = params.get("alpha[1]", np.nan)  # ARCH term
beta_1 = params.get("beta[1]", np.nan)  # GARCH term

# Number of periods for impulse response
n_irf_steps = 10  
irf = np.zeros(n_irf_steps)

# Compute response of volatility to a unit shock over time
shock = 1  # Initial shock
for i in range(n_irf_steps):
    irf[i] = shock * (alpha_1 * (beta_1 ** i))

# -----------------------------
# 4. Print Results & Decision
# -----------------------------
print("\n📌 Impulse Response Function (Flow 6):")
for i in range(n_irf_steps):
    print(f"   - Step {i+1}: {irf[i]:.6f}")

if np.all(irf < 1):
    print("\n   → Volatility response is decaying (Stable) ✅")
else:
    print("\n   → Volatility response is persistent (Unstable) ❌")

print("\n✅ Flow 6: Impulse Response Function Completed!\n")


📌 Impulse Response Function (Flow 6):
   - Step 1: 0.000000
   - Step 2: 0.000000
   - Step 3: 0.000000
   - Step 4: 0.000000
   - Step 5: 0.000000
   - Step 6: 0.000000
   - Step 7: 0.000000
   - Step 8: 0.000000
   - Step 9: 0.000000
   - Step 10: 0.000000

   → Volatility response is decaying (Stable) ✅

✅ Flow 6: Impulse Response Function Completed!



In [14]:
# Flow 7: Information Criteria
import numpy as np
import pandas as pd
from arch import arch_model

# -----------------------------
# 1. Generate Random Time-Series Data (Same as Main Flow)
# -----------------------------
np.random.seed(42)
size = 1000  # Length of time series
returns = np.random.normal(loc=0, scale=1, size=size)  # Simulated log returns
returns_series = pd.Series(returns, name="Returns")

# -----------------------------
# 2. Fit GARCH(1,1) Model (Refers to Main Flow)
# -----------------------------
garch_model = arch_model(returns_series, vol='Garch', p=1, q=1, dist='normal')
garch_results = garch_model.fit(disp="off")

# -----------------------------
# 3. Compute Model Selection Criteria (AIC & BIC)
# -----------------------------
aic = garch_results.aic  # Akaike Information Criterion
bic = garch_results.bic  # Bayesian Information Criterion

# -----------------------------
# 4. Print Results & Decision
# -----------------------------
print("\n📌 Information Criteria (Flow 7):")
print(f"   - Akaike Information Criterion (AIC): {aic:.6f}")
print(f"   - Bayesian Information Criterion (BIC): {bic:.6f}")

# Decision: Lower AIC/BIC → Better Model
if aic < bic:
    print("\n   → Model Selection Favored by AIC ✅")
else:
    print("\n   → Model Selection Favored by BIC ✅")

print("\n✅ Flow 7: Information Criteria Completed!\n")


📌 Information Criteria (Flow 7):
   - Akaike Information Criterion (AIC): 2802.468168
   - Bayesian Information Criterion (BIC): 2822.099189

   → Model Selection Favored by AIC ✅

✅ Flow 7: Information Criteria Completed!

